In [1]:
import pandas as pd 
import numpy as np

full_dataset = pd.read_csv("data/flights.csv")
full_dataset = full_dataset[np.isfinite(full_dataset['ARRIVAL_DELAY'])]

print (full_dataset.shape)

/Users/nnbenavides/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(5714008, 31)


In [2]:
#full_dataset.head()
sampled_data = full_dataset[full_dataset['ORIGIN_AIRPORT'] == 'ORD']
sampled_data.head()
print (sampled_data.shape)

(276554, 31)


In [6]:
sample_size = 100000
sampled_data = sampled_data.sample(sample_size)
print sampled_data.shape
sampled_data = sampled_data.reset_index(drop = True)
sampled_data.head()

(100000, 31)


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,6,30,2,EV,3936,N14198,ORD,FAR,1540,...,1721.0,-19.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,2,22,7,AA,2284,N3BAAA,ORD,DCA,2045,...,2330.0,2.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,12,4,5,EV,6034,N12900,ORD,ELM,2100,...,2340.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,9,13,7,OO,6213,N971SW,ORD,CMH,1105,...,1314.0,-6.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,9,3,4,EV,4758,N11187,ORD,MEM,930,...,1109.0,-12.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [8]:
baseline_features = sampled_data[['AIRLINE','MONTH', 'DAY_OF_WEEK','ORIGIN_AIRPORT', 'DESTINATION_AIRPORT','SCHEDULED_DEPARTURE']]

In [9]:
baseline_yx = sampled_data[['ARRIVAL_DELAY', 'AIRLINE','MONTH','DAY_OF_WEEK','ORIGIN_AIRPORT', 'DESTINATION_AIRPORT','SCHEDULED_DEPARTURE']]

In [ ]:
print 'starting update'
for i,row in baseline_yx.iterrows():
    if (i + 1) % (sample_size/10) == 0:
        print 'updated ' + str(i + 1)
    delay = baseline_yx.iat[i, 0]
    if delay < 0:
        baseline_yx.iat[i, 0] = 0
    elif delay <= 15:
        baseline_yx.iat[i, 0] = 0
    else:
        baseline_yx.iat[i, 0] = 1
    baseline_yx.iat[i, 6] = int(baseline_yx.iat[i,6]) / 100

starting update


In [ ]:
encoded_target = baseline_yx[['ARRIVAL_DELAY']]

In [ ]:
encoded_baseline_features = pd.concat([baseline_features.drop('AIRLINE', axis=1), pd.get_dummies(baseline_features['AIRLINE'])], axis=1)
encoded_baseline_features = pd.concat([encoded_baseline_features, pd.get_dummies(baseline_features['ORIGIN_AIRPORT'])], axis=1)
encoded_baseline_features = pd.concat([encoded_baseline_features, pd.get_dummies(baseline_features['DESTINATION_AIRPORT'])], axis=1)
encoded_baseline_features = encoded_baseline_features.drop('ORIGIN_AIRPORT', axis = 1)
encoded_baseline_features= encoded_baseline_features.drop('DESTINATION_AIRPORT', axis = 1)


In [ ]:
oracle_yx = sampled_data[['ARRIVAL_DELAY','DAY', 'MONTH', 'AIRLINE','ORIGIN_AIRPORT', 'DESTINATION_AIRPORT','SCHEDULED_DEPARTURE', 'DEPARTURE_DELAY']]

In [ ]:
encoded_oracle_features = pd.concat([encoded_baseline_features, sampled_data['DEPARTURE_DELAY']], axis=1)
train_xo, test_xo, train_yo, test_yo = train_test_split(encoded_oracle_features, encoded_target, train_size=0.8)
lr_oracle = LogisticRegression()
lr_oracle.fit(train_xo, train_yo)
lr_train_acc = metrics.accuracy_score(train_yo, lr_oracle.predict(train_xo))
lr_test_acc = metrics.accuracy_score(test_yo, lr_oracle.predict(test_xo))
print ('oracle train acc: ' + str(lr_train_acc))
print ('oracle test acc: ' + str(lr_test_acc))

pred_y_testo = lr_oracle.predict(test_xo)
pred_y_testo = [1 if pred_y_testo[i] > 1 else 0 for i in range(len(pred_y_testo))]
test_y_listo = test_yo.as_matrix().tolist()
test_y_binarizedo = [1 if test_y_listo[i][0] > 1 else 0 for i in range(len(test_y_listo))]
print ('oracle precision: ' + str(metrics.precision_score(test_y_binarizedo, pred_y_testo)))
print ('oracle recall: ' + str(metrics.recall_score(test_y_binarizedo, pred_y_testo)))

In [ ]:
train_xb = train_xo.drop('DEPARTURE_DELAY', axis=1)
train_yb = train_yo
test_xb = test_xo.drop('DEPARTURE_DELAY', axis = 1)
test_yb = test_yo

In [ ]:
lr = LogisticRegression()
lr.fit(train_xb, train_yb)
lr_train_acc = metrics.accuracy_score(train_yb, lr.predict(train_xb))
lr_test_acc = metrics.accuracy_score(test_yb, lr.predict(test_xb))
pred_y_test = list(lr.predict(test_xb))
pred_y_test = [1 if pred_y_test[i] > 1 else 0 for i in range(len(pred_y_test))]
test_y_list = test_y.as_matrix().tolist()
test_y_binarized = [1 if test_y_list[i][0] > 1 else 0 for i in range(len(test_y_list))]
print ('baseline train acc: ' + str(lr_train_acc))
print ('baseline test acc: ' + str(lr_test_acc))
print ('baseline precision: ' + str(metrics.precision_score(test_y_binarized, pred_y_test, average = 'binary')))

In [ ]:
y_true = test_y
y_pred = lr.predict(test_x)
confusion = confusion_matrix(y_true, y_pred)
print confusion